# TAAR – Evaluating existing recommenders

Not every recommender can always make a recommendation. To evaluate the individual recommenders for the ensemble, we want to find out how often this is the case and how well the recommenders complement each other.

This notebook either needs to be executed in the [TAAR](http://github.com/mozilla/taar) repository or somewhere where TAAR is in the Python path, because some TAAR recommenders are loaded in.

## Retrieving the relevant variables from the longitudinal dataset

In [105]:
%%time
frame = sqlContext.sql("""
WITH valid_clients AS (
    SELECT *
    FROM longitudinal
    WHERE normalized_channel='release' AND build IS NOT NULL AND build[0].application_name='Firefox'
),

addons AS (
    SELECT client_id, feature_row.*
    FROM valid_clients
    LATERAL VIEW explode(active_addons[0]) feature_row
),
    
non_system_addons AS(
    SELECT client_id, collect_set(key) AS installed_addons
    FROM addons
    WHERE NOT value.is_system
    GROUP BY client_id
)

SELECT
    l.client_id,
    non_system_addons.installed_addons,
    settings[0].locale AS locale,
    geo_city[0] AS geoCity,
    subsession_length[0] AS subsessionLength,
    system_os[0].name AS os,
    scalar_parent_browser_engagement_total_uri_count[0].value AS total_uri,
    scalar_parent_browser_engagement_tab_open_event_count[0].value as tab_open_count,
    places_bookmarks_count[0].sum as bookmark_count,
    scalar_parent_browser_engagement_unique_domains_count[0].value as unique_tlds,
    profile_creation_date[0] as profile_date,
    submission_date[0] as submission_date
FROM valid_clients l LEFT OUTER JOIN non_system_addons
ON l.client_id = non_system_addons.client_id
""")

rdd = frame.rdd

CPU times: user 140 ms, sys: 12 ms, total: 152 ms
Wall time: 18min 43s


## Loading addon data (AMO)

We need to load the addon database to find out which addons are legacy addons.

In [106]:
from taar.recommenders.utils import get_s3_json_content

In [107]:
AMO_DUMP_BUCKET = 'telemetry-parquet'
AMO_DUMP_KEY = 'telemetry-ml/addon_recommender/addons_database.json'

In [108]:
amo_dump = get_s3_json_content(AMO_DUMP_BUCKET, AMO_DUMP_KEY)

## Filtering out legacy addons

This is a helper function that takes a list of addon IDs and only returns the IDs that are from legacy addons.

In [109]:
def get_legacy_addons(installed_addons):
    legacy_addons = []
    
    for addon_id in installed_addons:
        if addon_id in amo_dump:
            addon = amo_dump[addon_id]
            addon_files = addon.get('current_version', {}).get('files', {})

            is_webextension = any([f.get("is_webextension", False) for f in addon_files])
            is_legacy = not is_webextension

            if is_legacy:
                legacy_addons.append(addon_id)
            
    return legacy_addons

## Completing client data

In [110]:
from dateutil.parser import parse as parse_date
from datetime import datetime

In [111]:
def compute_weeks_ago(formatted_date):
    try:
        date = parse_date(formatted_date).replace(tzinfo=None)
        days_ago = (datetime.today() - date).days
        return days_ago / 7
    except:
        return float("inf")

In [391]:
def complete_client_data(client_data):
    client = client_data.asDict()
    
    client['installed_addons'] = client['installed_addons'] or []
    client['disabled_addon_ids'] = get_legacy_addons(client['installed_addons'])
    client['locale'] = str(client['locale'])
    client['profile_age_in_weeks'] = compute_weeks_ago(client['profile_date'])
    client['submission_age_in_weeks'] = compute_weeks_ago(client['submission_date'])
    
    return client

## Evaluating the existing recommenders

To check if a recommender is able to make a recommendation, it's sometimes easier and cleaner to directly query it instead of checking the important attributes ourselves. For example, this is the case for the locale recommender.

In [113]:
from taar.recommenders import CollaborativeRecommender, LegacyRecommender, LocaleRecommender

In [114]:
class DummySimilarityRecommender:
    def can_recommend(self, client_data):
        REQUIRED_FIELDS = ["geoCity", "subsessionLength", "locale", "os", "bookmark_count", "tab_open_count",
                           "total_uri", "unique_tlds"]

        has_fields = all([client_data.get(f, None) is not None for f in REQUIRED_FIELDS])
        return has_fields

In [115]:
recommenders = {
    "collaborative": CollaborativeRecommender(),
    "legacy": LegacyRecommender(),
    "locale": LocaleRecommender(),
    "similarity": DummySimilarityRecommender()
}

In [116]:
def test_recommenders(client):
    return tuple([recommender.can_recommend(client) for recommender in recommenders.values()])

## Computing combined counts

We iterate over all clients in the longitudinal dataset, change the attributes to the expected format and then query the individual recommenders.

In [117]:
from operator import add
from collections import defaultdict

In [392]:
rdd_completed = rdd.map(complete_client_data)

In [119]:
def analyse(rdd):
    results = rdd\
        .map(test_recommenders)\
        .map(lambda x: (x, 1))\
        .reduceByKey(add)\
        .collect()
        
    return defaultdict(int, results)

In [349]:
%time results = analyse(rdd_completed)

CPU times: user 9.71 s, sys: 404 ms, total: 10.1 s
Wall time: 13min 10s


In [351]:
num_clients = sum(results.values())
total_results = results

## Computing individual counts

In [249]:
individual_counts = []

for i in range(len(recommenders)):
    count = 0
    
    for key, key_count in results.items():
        if key[i]:
            count += key_count
            
    individual_counts.append(count)

## Displaying the results

In [250]:
from pandas import DataFrame

In [251]:
def format_int(num):
    return "{:,}".format(num)

In [252]:
def format_frequency(frequency):
    return "%.5f" % frequency

In [253]:
def get_relative_counts(counts, total=num_clients):
    return [format_frequency(count / float(total)) for count in counts]

This is a bit hacky. Sorting a data frame by formatted counts does not work; so we have to add the unformatted ones, sort the data frame, and then remove that column again.

In [254]:
def sorted_dataframe(df, order, key="unformatted_counts"):
    df[key] = order
    return df.sort_values(by=key, ascending=False).drop(key, axis=1)

### Individual counts

In [255]:
df = DataFrame(index=recommenders.keys(),
          columns=["Relative count"],
          data=get_relative_counts(individual_counts)
)

sorted_dataframe(df, individual_counts)

,Relative count
locale,0.99977
collaborative,0.41949
similarity,0.28339
legacy,0.01575


$\implies$ The locale and collaborative recommenders are able to generate recommendations most of the time. The legacy recommender can only make recommendations very seldomly as not many users seem to have (legacy) addons installed.

### Combined counts

It's interesting to see how well the individual recommenders complement each other. In the following, we count how often different combinations of the recommenders can make recommendations.

The table is easier to read if cells are empty if a recommender is not available. If this is not desired, these variables can be changed:

In [256]:
recommender_available_label = "Available"
recommender_unavailable_label = ""

In [257]:
def format_labels(keys):
    return tuple([recommender_available_label if key else recommender_unavailable_label for key in keys])

In [258]:
def format_data(keys, counts):
    formatted_keys = map(format_labels, keys)
    return [elems + count for elems, count in zip(formatted_keys, zip(*counts))]

In [259]:
columns = recommenders.keys() + ["Relative counts"]

counts = get_relative_counts(results.values())
data = format_data(results.keys(), [counts])

In [260]:
df = DataFrame(columns=columns, data=data)
sorted_dataframe(df, results.values())

,locale,legacy,collaborative,similarity,Relative counts
1,Available,,,,0.44747
3,Available,,Available,,0.26032
0,Available,,Available,Available,0.14333
5,Available,,,Available,0.13290
10,Available,Available,Available,,0.00865
7,Available,Available,Available,Available,0.00710
2,,,,,0.00011
6,,,Available,,0.00006
9,,,Available,Available,0.00003
11,,,,Available,0.00003


$\implies$ If any recommender is available, then the locale recommenders is generally also available. Other than that, there is a good chance the the collaborative recommender is available.
There is only a very small portion of cases where the similarity recommender was able to make a recommendation, when locale/collaborative were not; and not a single such case for the legacy recommender.

### Grouped by number of available recommenders

In [261]:
from itertools import groupby
from operator import itemgetter

In [262]:
from IPython.display import display, Markdown

In [263]:
for num, group in groupby(sorted(results.keys(), key=sum), sum):
    display(Markdown("#### %d available recommender%s" % (num, "s" if num != 1 else "")))
    
    sub_keys = list(group)
    formatted_keys = map(format_labels, sub_keys)
    
    sub_counts = [results[key] for key in sub_keys]
    sub_counts_to_total = get_relative_counts(sub_counts)
    sub_counts_to_table = get_relative_counts(sub_counts, sum(sub_counts))
    
    zipped_data = zip(formatted_keys, sub_counts_to_total, sub_counts_to_table)
    data = [elems + (counts, table_counts) for elems, counts, table_counts in zipped_data]
    
    columns = recommenders.keys() + ["Relative to all", "Relative to this table"]
    
    df = DataFrame(columns=columns, data=data)
    df = sorted_dataframe(df, sub_counts)
    display(df)

#### 0 available recommenders

,locale,legacy,collaborative,similarity,Relative to all,Relative to this table
0,,,,,0.00011,1.00000


#### 1 available recommender

,locale,legacy,collaborative,similarity,Relative to all,Relative to this table
0,Available,,,,0.44747,0.99980
1,,,Available,,0.00006,0.00014
2,,,,Available,0.00003,0.00006


#### 2 available recommenders

,locale,legacy,collaborative,similarity,Relative to all,Relative to this table
0,Available,,Available,,0.26032,0.66197
1,Available,,,Available,0.13290,0.33795
3,,,Available,Available,0.00003,0.00007
2,,Available,Available,,0.00000,0.00000


#### 3 available recommenders

,locale,legacy,collaborative,similarity,Relative to all,Relative to this table
0,Available,,Available,Available,0.14333,0.94310
2,Available,Available,Available,,0.00865,0.05690
1,,Available,Available,Available,0.00000,0.00001


#### 4 available recommenders

,locale,legacy,collaborative,similarity,Relative to all,Relative to this table
0,Available,Available,Available,Available,0.00710,1.00000


## By dates

In this section, we perform a similar analysis as before but on subsets of the data. These subsets are specified by when the client profile was generated. `conditions` is a list that contains ranges for the profile age in weeks. The end of the range is exclusive, similar to ranges in Python's standard library.

In [137]:
conditions = [
    (0, 1),
    (1, 2),
    (2, 3),
    (3, 4)
]

In [303]:
import numpy as np
from numpy import argsort
from itertools import product

In [139]:
def attribute_between(attr, min_weeks, max_weeks):
    return lambda client: min_weeks <= client[attr] < max_weeks

In [369]:
def get_conditioned_results(attr, conditions):
    conditioned_results = {}

    for (min_weeks, max_weeks) in conditions:
        sub_rdd = rdd_completed.filter(attribute_between(attr, min_weeks, max_weeks))
        conditioned_results[(min_weeks, max_weeks)] = analyse(sub_rdd)
        
    return conditioned_results

### By profile age in weeks

In [140]:
%time conditioned_results = get_conditioned_results("profile_age_in_weeks", conditions)

CPU times: user 39.1 s, sys: 1.49 s, total: 40.6 s
Wall time: 52min 19s


To make things a little bit easier to read, only recommender combinations that actually appear are displayed in the table.

In [402]:
def nonzero_combinations(conditioned_results):
    combinations = []

    for sub_result in conditioned_results.values():
        combinations += [key for key, value in sub_result.items() if value > 0]

    return set(combinations)

In [410]:
combinations = nonzero_combinations(conditioned_results)

In [411]:
def display_individual_filtered_results(conditioned_results, combinations, label):
    display(Markdown("### Filtering on the %s, Python-like exclusive ranges" % label))

    counts = []
    titles = []

    columns = recommenders.keys() + ["Relative counts"]

    for key in conditions:
        sub_results = conditioned_results[key]
        values = [sub_results[sub_key] for sub_key in combinations]
        summed = sum(values)

        sub_counts = get_relative_counts(values, summed)
        data = format_data(combinations, [sub_counts])
        counts.append(sub_counts)

        title = "Between %d and %d weeks" % key
        titles.append(title)
        display(Markdown("#### %s" % title))

        df = DataFrame(columns=columns, data=data)
        df = sorted_dataframe(df, values)
        display(df)

    return counts, titles

In [419]:
counts, titles = display_individual_filtered_results(conditioned_results, combinations, label="profile age")

### Filtering on the profile age, Python-like exclusive ranges

#### Between 0 and 1 weeks

,locale,legacy,collaborative,similarity,Relative counts
1,Available,,,,0.56426
3,Available,,Available,,0.24668
5,Available,,,Available,0.11291
0,Available,,Available,Available,0.06858
10,Available,Available,Available,,0.00447
7,Available,Available,Available,Available,0.00304
8,,,,,0.00006
2,,Available,Available,,0.00000
4,,Available,Available,Available,0.00000
6,,,Available,,0.00000


#### Between 1 and 2 weeks

,locale,legacy,collaborative,similarity,Relative counts
1,Available,,,,0.55178
3,Available,,Available,,0.21705
5,Available,,,Available,0.14102
0,Available,,Available,Available,0.08284
10,Available,Available,Available,,0.00407
7,Available,Available,Available,Available,0.00307
6,,,Available,,0.00008
8,,,,,0.00008
9,,,Available,Available,0.00001
2,,Available,Available,,0.00000


#### Between 2 and 3 weeks

,locale,legacy,collaborative,similarity,Relative counts
1,Available,,,,0.52717
3,Available,,Available,,0.22324
5,Available,,,Available,0.14923
0,Available,,Available,Available,0.09279
10,Available,Available,Available,,0.00400
7,Available,Available,Available,Available,0.00344
8,,,,,0.00006
6,,,Available,,0.00003
9,,,Available,Available,0.00003
11,,,,Available,0.00001


#### Between 3 and 4 weeks

,locale,legacy,collaborative,similarity,Relative counts
1,Available,,,,0.52334
3,Available,,Available,,0.22640
5,Available,,,Available,0.14447
0,Available,,Available,Available,0.09773
10,Available,Available,Available,,0.00421
7,Available,Available,Available,Available,0.00375
8,,,,,0.00006
6,,,Available,,0.00003
9,,,Available,Available,0.00002
2,,Available,Available,,0.00000


To make things a little bit easier to read, we can display all results in a single table.

In [406]:
def display_merged_filtered_results(counts, titles, total_results, combinations, label):
    values = [total_results[sub_key] for sub_key in combinations]
    sub_counts = get_relative_counts(values)
    counts.append(sub_counts)
    titles.append("Total, without any condition")  

    columns = recommenders.keys() + titles
    data = format_data(combinations, counts)

    df = DataFrame(columns=columns, data=data)
    df = sorted_dataframe(df, counts[0])

    display(Markdown("### Filtering on the %s, Python-like exclusive ranges – All in one table" % label))
    display(df)

In [407]:
display_merged_filtered_results(counts, titles, total_results, combinations, label="profile age")

### Filtering on the profile age, Python-like exclusive ranges – All in one table

,locale,legacy,collaborative,similarity,Between 0 and 1 weeks,Between 1 and 2 weeks,Between 2 and 3 weeks,Between 3 and 4 weeks,"Total, without any condition"
9,Available,,,,0.56426,0.55178,0.52717,0.52334,0.44747
2,Available,,Available,,0.24668,0.21705,0.22324,0.22640,0.26032
4,Available,,,Available,0.11291,0.14102,0.14923,0.14447,0.13290
0,Available,,Available,Available,0.06858,0.08284,0.09279,0.09773,0.14333
1,Available,Available,Available,,0.00447,0.00407,0.00400,0.00421,0.00865
6,Available,Available,Available,Available,0.00304,0.00307,0.00344,0.00375,0.00710
7,,,,,0.00006,0.00008,0.00006,0.00006,0.00011
3,,,,Available,0.00000,0.00000,0.00001,0.00000,0.00003
5,,,Available,,0.00000,0.00008,0.00003,0.00003,0.00006
8,,,Available,Available,0.00000,0.00001,0.00003,0.00002,0.00003


### By submission date in weeks

In [ ]:
%time conditioned_results_submission_date = get_conditioned_results("submission_age_in_weeks", conditions)

In [423]:
label = "submission date"
combinations = nonzero_combinations(conditioned_results_submission_date)
counts, titles = display_individual_filtered_results(conditioned_results_submission_date, combinations, label)
display_merged_filtered_results(counts, titles, total_results, combinations, label)

### Filtering on the submission date, Python-like exclusive ranges

#### Between 0 and 1 weeks

,locale,legacy,collaborative,similarity,Relative counts
1,Available,,,,0.28043
0,Available,,Available,Available,0.25749
3,Available,,Available,,0.23062
5,Available,,,Available,0.20527
7,Available,Available,Available,Available,0.01489
10,Available,Available,Available,,0.01114
8,,,,,0.00005
11,,,,Available,0.00004
6,,,Available,,0.00003
9,,,Available,Available,0.00003


#### Between 1 and 2 weeks

,locale,legacy,collaborative,similarity,Relative counts
1,Available,,,,0.41392
3,Available,,Available,,0.25433
5,Available,,,Available,0.15828
0,Available,,Available,Available,0.15660
10,Available,Available,Available,,0.00968
7,Available,Available,Available,Available,0.00701
8,,,,,0.00009
6,,,Available,,0.00004
9,,,Available,Available,0.00003
11,,,,Available,0.00003


...

,locale,legacy,collaborative,similarity,Relative counts
1,Available,,,,0.46368
3,Available,,Available,,0.26353
5,Available,,,Available,0.13141
0,Available,,Available,Available,0.12639
10,Available,Available,Available,,0.00911
7,Available,Available,Available,Available,0.00568
8,,,,,0.00011
6,,,Available,,0.00005
9,,,Available,Available,0.00002
11,,,,Available,0.00002


#### Between 3 and 4 weeks

,locale,legacy,collaborative,similarity,Relative counts
1,Available,,,,0.48052
3,Available,,Available,,0.26312
5,Available,,,Available,0.12283
0,Available,,Available,Available,0.11945
10,Available,Available,Available,,0.00870
7,Available,Available,Available,Available,0.00514
8,,,,,0.00015
6,,,Available,,0.00003
11,,,,Available,0.00003
9,,,Available,Available,0.00002


### Filtering on the submission date, Python-like exclusive ranges – All in one table

,locale,legacy,collaborative,similarity,Between 0 and 1 weeks,Between 1 and 2 weeks,Between 2 and 3 weeks,Between 3 and 4 weeks,"Total, without any condition"
1,Available,,,,0.28043,0.41392,0.46368,0.48052,0.44747
0,Available,,Available,Available,0.25749,0.15660,0.12639,0.11945,0.14333
3,Available,,Available,,0.23062,0.25433,0.26353,0.26312,0.26032
5,Available,,,Available,0.20527,0.15828,0.13141,0.12283,0.13290
7,Available,Available,Available,Available,0.01489,0.00701,0.00568,0.00514,0.00710
10,Available,Available,Available,,0.01114,0.00968,0.00911,0.00870,0.00865
8,,,,,0.00005,0.00009,0.00011,0.00015,0.00011
11,,,,Available,0.00004,0.00003,0.00002,0.00003,0.00003
6,,,Available,,0.00003,0.00004,0.00005,0.00003,0.00006
9,,,Available,Available,0.00003,0.00003,0.00002,0.00002,0.00003
